In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
posts = pd.read_csv('../data/posts.csv')
users = pd.read_csv('../data/users_train.csv')
users_test = pd.read_csv('../data/users_test.csv')
friends = pd.read_csv('../data/friends.csv')

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


def train(users, posts):
    valid_users = np.unique(users['uid'])
    posts = posts[posts['from_id'].isin(valid_users)]
    posts['text'] = posts['text'].apply(str)
    data = posts.groupby(['from_id'], sort=True).agg({'text': ' '.join,
                                                      'comments': 'sum',
                                                      'likes': 'sum',
                                                      'reposts': 'sum'})
    users.index = users.uid
    users.drop('uid', axis=1, inplace=True)
    data['is_leader'] = users.loc[data.index]['is_leader']
    X = data.drop('is_leader', axis=1)
    Y = data['is_leader'].values
    
    tfidf = TfidfVectorizer()
    model = LogisticRegression()

    model.fit(tfidf.fit_transform(X['text']), Y)
    return model, tfidf

In [4]:
def predict(users, posts, model, tfidf):
    valid_users = np.unique(users['uid'])
    posts = posts[posts['from_id'].isin(valid_users)]
    posts['text'] = posts['text'].apply(str)
    data = posts.groupby(['from_id'], sort=True).agg({'text': ' '.join,
                                                      'comments': 'sum',
                                                      'likes': 'sum',
                                                      'reposts': 'sum'})
    users.index = users.uid
    users.drop('uid', axis=1, inplace=True)
    return model.predict(tfidf.transform(data['text']))

In [5]:
model, tfidf = train(users.copy(), posts.copy())

/Users/max/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
preds = predict(users_test.copy(), posts.copy(), model, tfidf)

/Users/max/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [9]:
# submission = pd.DataFrame({'is_leader':preds}, index=X.index)

# submission.to_csv('../data/train_submission.csv')